In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import geopandas as gpd
import random
from collections import deque

In [10]:
# path_to_counties_dir = "2020/{0}/counties/"
# path_to_dist_csv = path_to_counties_dir + "graph/{0}_distances.csv"
# path_to_feature_csv = path_to_counties_dir + "{0}_features.csv"

# county = "OK"
dist_df = pd.read_csv('AK_distances.csv')
feature_df = pd.read_csv('AK_features.csv', header=None)
pop_df = feature_df[[1]]
pop_df.columns = ['pop']
pop_df["id"] = [i for i in range(len(pop_df))]
pop_df.head()

zero_matrix = np.zeros((len(feature_df), len(feature_df)))
adj_df = pd.DataFrame(zero_matrix)
id_map = {}
for i, id in enumerate(feature_df[0]):
    id_map[id] = i

for node, adj_list in zip(feature_df[0], feature_df[2]):
    node_idx = id_map[node]
    for adj in adj_list.split(','):
        adj_idx = id_map[int(str(adj).lstrip("0"))]
        adj_df.iloc[node_idx, adj_idx] = 1

G = nx.from_pandas_adjacency(adj_df)
G.edges()

FileNotFoundError: [Errno 2] No such file or directory: 'AK_distances.csv'

In [2]:
def get_boundary_nodes(districts, G):
    boundary_nodes = []
    for node in G.nodes():
        node_district = districts[node]
        for neighbor in G.neighbors(node):
            if districts[neighbor] != node_district:
                boundary_nodes.append(node)
                break
    return boundary_nodes

In [3]:
def generate_neighbor(districts, G, num_changes=1):
    for _ in range(num_changes):
        boundary_nodes = get_boundary_nodes(districts, G)
        if not boundary_nodes:
            return districts

        node_to_change = random.choice(boundary_nodes)
        # Check if the node to change is the only node in a district, then generate a new node to change.
        while len(np.where(np.array(districts) == districts[node_to_change])[0]) == 1:
            node_to_change = random.choice(boundary_nodes)

        current_district = districts[node_to_change]
        neighbor_districts = {districts[n] for n in G.neighbors(node_to_change) if districts[n] != current_district and districts[n]}
        
        if not neighbor_districts:
            # If there are no neighbor districts to switch to, return the current districts
            return districts

        new_district = random.choice(list(neighbor_districts))
        # new_districts = districts.copy()
        districts[node_to_change] = new_district
    return districts

In [4]:
def get_initial_solution(G, K):
    G_copy = deepcopy(G)
    node_list = list(G_copy.nodes)
    edge_list = list(G_copy.edges)
    dist_assignments = [0]*len(node_list)
    nodes_to_visit = deque()
    visited = []
    for k in range(K):
        selected_node = random.choice(node_list)
        node_list.remove(selected_node)
        dist_assignments[selected_node] = k
        nodes_to_visit.append(selected_node)
        visited.append(selected_node)
    # print(Counter(dist_assignments))
    while nodes_to_visit:
        node = nodes_to_visit.popleft()
        adj_list = list(G_copy.edges(node))
        for i, edge in enumerate(deepcopy(adj_list)):
            if edge[1] in visited:
                adj_list.remove(edge)
                G_copy.remove_edge(*edge)
                
        if len(adj_list) == 0:
            continue
        edge = random.choice(adj_list)
        G_copy.remove_edge(*edge)
        node_dist = dist_assignments[node]
        dist_assignments[edge[1]] = node_dist
        visited.append(edge[1])
        # print("Appending node", edge[1])
        nodes_to_visit.append(edge[1])
    return dist_assignments

In [5]:
def calculate_district_populations(solution, G, K):
    district_populations = {i: 0 for i in range(K)}
    for i, node in enumerate(list(G.nodes())):
        k_assignment = solution[i]
        district_populations[k_assignment] += pop_df.iloc[node]['pop']
    return district_populations

In [6]:
def is_contiguous(cluster_assignments, k, G):
    """
    Check if a cluster is contiguous in the Queen's case.
    cluster: a list of tuples representing the coordinates of the cells in the cluster.
    """
    visited = set()
    edge_list = list(G.edges())
    cluster_edges = [edge_list[i] for i in np.where(cluster_assignments == k)[0]]
    to_visit = [cluster_edges[0]]  # Start with the first cell

    while to_visit:
        cell = to_visit.pop()
        if cell in visited:
            continue
        visited.add(cell)
        neighbors = (cell)  # Implement this function based on your grid
        # Add neighbors that are in the cluster and not visited yet to the queue
        to_visit.extend([n for n in neighbors if n in cluster_edges and n not in visited])

    return len(visited) == len(cluster_edges)

In [7]:
def calculate_solution_score(solution, G, contiguity_penalty, K):
    district_populations = calculate_district_populations(solution, G, K)
    # contiguity_penalty = contiguity_penalty
    contiguity_score = 0
    for i, k in enumerate(range(K)):
        if not is_contiguous(np.array(solution), k, G):
            contiguity_score += contiguity_penalty
    popultation_score = 0
    for k, pop in district_populations.items():
        popultation_score += abs(pop - ideal_population)
    return contiguity_score + popultation_score

1. Initial Population
2. Fitness Function
3. Selection
4. Crossover
5. Mutation

In [8]:
# 1. Initial Population
def create_initial_pop(size, G, K):

    population = []
    for i in range(size):
        member = get_initial_solution(G, K)
        population.append(member)

    return population

In [9]:
K = 5
G = nx.from_pandas_adjacency(adj_df)


create_initial_pop(10, )

NameError: name 'adj_df' is not defined